# Figuring out what determines the price of HDB resale flats in Singapore


### Part 1 Set Up

Import the necessary libraries and set up credentials for Foursquare API

In [2]:
import pandas as pd
import numpy as np
import folium
import json
import requests
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.distance import distance

import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
CLIENT_ID = '2ZASQFQMO4SC15PPC0B2IWDCD4RKQFL0MLPH2XFA4DNNO4ZD' # your Foursquare ID
CLIENT_SECRET = 'PP2HSIHJFG1AUFB5PH5KIGMNVZD5HAWIHRR5KKA0PHHCZ2HT' # your Foursquare Secret
VERSION = '20201216' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2ZASQFQMO4SC15PPC0B2IWDCD4RKQFL0MLPH2XFA4DNNO4ZD
CLIENT_SECRET:PP2HSIHJFG1AUFB5PH5KIGMNVZD5HAWIHRR5KKA0PHHCZ2HT


### Part 2 - Data Preprocessing

1. Get the coordinates for Singapore for the map
2. Get the coordinates for MRT Stations

In [4]:
address = 'Singapore Singapore'

geolocator = Nominatim(user_agent="singapore_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.2904753, 103.8520359.


In [5]:
mrt_df = pd.read_csv('mrtsg.csv')
mrt_df = mrt_df[mrt_df['COLOR'] != 'OTHERS']

mrt_df.head()

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
5,153,BARTLEY MRT STATION,CC12,33168.3039,36108.7003,1.342828,103.879746,YELLOW
7,115,BAYFRONT MRT STATION,DT16,30867.0093,29368.6250,1.281874,103.859073,BLUE


In [6]:
# Plot address onto the Map

map_singapore = folium.Map(location=[latitude, longitude], zoom_start=11)

map_singapore

for lng, lat, station_name in zip(mrt_df['Longitude'], mrt_df['Latitude'], mrt_df['STN_NAME']):
    label = f'Station Name: {station_name}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_singapore)
    
map_singapore

#### Importing the dataset for resale flat prices

2 new columns are created to be used for geopy reverse coordinates lookup

In [7]:
hdb_resale_df = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

hdb_resale_df['street_name_address'] = hdb_resale_df['street_name'] + ', SINGAPORE'
hdb_resale_df['town_address'] = hdb_resale_df['town'] + ', SINGAPORE'

hdb_resale_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street_name_address,town_address
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,"ANG MO KIO AVE 10, SINGAPORE","ANG MO KIO, SINGAPORE"
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,"ANG MO KIO AVE 4, SINGAPORE","ANG MO KIO, SINGAPORE"
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,"ANG MO KIO AVE 5, SINGAPORE","ANG MO KIO, SINGAPORE"
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,"ANG MO KIO AVE 10, SINGAPORE","ANG MO KIO, SINGAPORE"
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,"ANG MO KIO AVE 5, SINGAPORE","ANG MO KIO, SINGAPORE"


In [8]:
street_address_df = hdb_resale_df.groupby(['street_name_address'])['block'].count().to_frame().reset_index()
town_address_df = hdb_resale_df.groupby(['town_address'])['block'].count().to_frame().reset_index()

print(street_address_df.head())

print(town_address_df.head())

         street_name_address  block
0    ADMIRALTY DR, SINGAPORE    372
1  ADMIRALTY LINK, SINGAPORE    295
2      AH HOOD RD, SINGAPORE     13
3   ALJUNIED CRES, SINGAPORE    147
4     ALJUNIED RD, SINGAPORE     14
             town_address  block
0   ANG MO KIO, SINGAPORE   3698
1        BEDOK, SINGAPORE   4489
2       BISHAN, SINGAPORE   1586
3  BUKIT BATOK, SINGAPORE   3019
4  BUKIT MERAH, SINGAPORE   3194


#### Get the coordinates of the houses based on Street Address / Town Address

Geopy is used and each row is looped to get the coordinates

In [10]:
from geopy.extra.rate_limiter import RateLimiter

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.1)

street_address_df['location'] = street_address_df['street_name_address'].apply(geocode)
street_address_df['point'] = street_address_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
street_address_df[['latitude', 'longitude', 'Altitude']] = pd.DataFrame(street_address_df['point'].tolist(), 
                                                                        index=street_address_df.index)

town_address_df['location'] = town_address_df['town_address'].apply(geocode)
town_address_df['point'] = town_address_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
town_address_df[['latitude', 'longitude', 'Altitude']] = pd.DataFrame(town_address_df['point'].tolist(), 
                                                                        index=town_address_df.index)

In [11]:
# rename columns

street_address_df.rename(columns={'latitude': 'street_latitude', 
                                  'longitude': 'street_longitude'},
                        inplace=True)

town_address_df.rename(columns={'latitude': 'town_latitude', 
                                  'longitude': 'town_longitude'},
                        inplace=True)

#### Ensure that all rows have longitude and latitude data

Some of the street name address is not searchable via geopy. To mitigate this, we use the town address as a lookup backup. This can be seen under the np.where section

In [12]:
# join back to master df

hdb_resale_df = hdb_resale_df.merge(street_address_df[['street_name_address', 'street_latitude', 'street_longitude']], how='left', left_on='street_name_address', right_on='street_name_address')

hdb_resale_df = hdb_resale_df.merge(town_address_df[['town_address', 'town_latitude', 'town_longitude']], how='left', left_on='town_address', right_on='town_address')


# ensure that all rows have latitude and longitude
hdb_resale_df['latitude'] = np.where(hdb_resale_df['street_latitude'].isnull(),
                                     hdb_resale_df['town_latitude'],
                                     hdb_resale_df['street_latitude'])

hdb_resale_df['longitude'] = np.where(hdb_resale_df['street_longitude'].isnull(),
                                     hdb_resale_df['town_longitude'],
                                     hdb_resale_df['street_longitude'])

hdb_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street_name_address,town_address,street_latitude,street_longitude,town_latitude,town_longitude,latitude,longitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,"ANG MO KIO AVE 10, SINGAPORE","ANG MO KIO, SINGAPORE",1.367602,103.856844,1.370080,103.849523,1.367602,103.856844
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,"ANG MO KIO AVE 4, SINGAPORE","ANG MO KIO, SINGAPORE",1.382319,103.838706,1.370080,103.849523,1.382319,103.838706
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,"ANG MO KIO AVE 5, SINGAPORE","ANG MO KIO, SINGAPORE",1.376825,103.847556,1.370080,103.849523,1.376825,103.847556
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,"ANG MO KIO AVE 10, SINGAPORE","ANG MO KIO, SINGAPORE",1.367602,103.856844,1.370080,103.849523,1.367602,103.856844
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,"ANG MO KIO AVE 5, SINGAPORE","ANG MO KIO, SINGAPORE",1.376825,103.847556,1.370080,103.849523,1.376825,103.847556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82796,2020-10,YISHUN,EXECUTIVE,834,YISHUN ST 81,04 TO 06,146.0,Maisonette,1988,66 years 03 months,680000.0,"YISHUN ST 81, SINGAPORE","YISHUN, SINGAPORE",1.415253,103.837021,1.429384,103.835028,1.415253,103.837021
82797,2020-10,YISHUN,EXECUTIVE,877,YISHUN ST 81,01 TO 03,145.0,Maisonette,1987,66 years 02 months,618000.0,"YISHUN ST 81, SINGAPORE","YISHUN, SINGAPORE",1.415253,103.837021,1.429384,103.835028,1.415253,103.837021
82798,2020-10,YISHUN,EXECUTIVE,877,YISHUN ST 81,07 TO 09,142.0,Apartment,1987,66 years 02 months,670000.0,"YISHUN ST 81, SINGAPORE","YISHUN, SINGAPORE",1.415253,103.837021,1.429384,103.835028,1.415253,103.837021
82799,2020-10,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,66 years 02 months,750000.0,"YISHUN ST 61, SINGAPORE","YISHUN, SINGAPORE",1.418095,103.838668,1.429384,103.835028,1.418095,103.838668


In [13]:
hdb_resale_df_final = hdb_resale_df[['month', 'town', 'flat_type', 'block',
                                     'street_name', 'storey_range', 'floor_area_sqm',
                                      'flat_model', 'lease_commence_date', 'remaining_lease',
                                      'resale_price', 'latitude', 'longitude']]

hdb_resale_df_final

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,latitude,longitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,1.367602,103.856844
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,1.382319,103.838706
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,1.376825,103.847556
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,1.367602,103.856844
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,1.376825,103.847556
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82796,2020-10,YISHUN,EXECUTIVE,834,YISHUN ST 81,04 TO 06,146.0,Maisonette,1988,66 years 03 months,680000.0,1.415253,103.837021
82797,2020-10,YISHUN,EXECUTIVE,877,YISHUN ST 81,01 TO 03,145.0,Maisonette,1987,66 years 02 months,618000.0,1.415253,103.837021
82798,2020-10,YISHUN,EXECUTIVE,877,YISHUN ST 81,07 TO 09,142.0,Apartment,1987,66 years 02 months,670000.0,1.415253,103.837021
82799,2020-10,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,10 TO 12,164.0,Multi Generation,1987,66 years 02 months,750000.0,1.418095,103.838668


#### Get distance of house to the nearest MRT (Train) Station

To get the distance, we use geopy's distance function and map every single house to every MRT station using cross join. After doing so, we get the row with the lowest distance (i.e. the nearest MRT Station)

In [14]:
def get_distance_between_points(lat_1, long_1, lat_2, long_2):
    coords_1 = (lat_1, long_1)
    coords_2 = (lat_2, long_2)

    distance_m = distance(coords_1, coords_2).m
    return distance_m

In [15]:
hdb_resale_df_final['cross_join_key'] = 1
mrt_df['cross_join_key'] = 1

combined_df = pd.merge(hdb_resale_df_final, mrt_df[['STN_NAME', 'cross_join_key', 'Latitude', 'Longitude']], on='cross_join_key')

combined_df['distance_to_mrt'] = combined_df[['latitude', 'longitude', 'Latitude', 'Longitude']].apply(lambda x: get_distance_between_points(*x), axis=1)

C:\Users\jimmy\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
idx = combined_df.groupby(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 
                     'floor_area_sqm', 'flat_model', 'remaining_lease', 
                     'resale_price', 'latitude', 'longitude'])['distance_to_mrt'].transform(min) == combined_df['distance_to_mrt']

hdb_resale_df_mrt = combined_df[idx]

hdb_resale_df_mrt.drop_duplicates(inplace=True)

hdb_resale_df_mrt.head()

C:\Users\jimmy\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,latitude,longitude,cross_join_key,STN_NAME,Latitude,Longitude,distance_to_mrt
2,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,1.367602,103.856844,1,ANG MO KIO MRT STATION,1.369933,103.849553,851.339283
276,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,1.382319,103.838706,1,YIO CHU KANG MRT STATION,1.381756,103.844944,696.968500
415,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,1.376825,103.847556,1,YIO CHU KANG MRT STATION,1.381756,103.844944,617.913682
419,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,1.367602,103.856844,1,ANG MO KIO MRT STATION,1.369933,103.849553,851.339283
693,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,1.376825,103.847556,1,YIO CHU KANG MRT STATION,1.381756,103.844944,617.913682


As it takes ages to run the code, we save the output into a file to ensure that it is easy to read again when needed

In [ ]:
hdb_resale_df_mrt.to_csv('hdb_map_to_nearest_mrt.csv')


In [3]:
hdb_resale_df_mrt = pd.read_csv('hdb_map_to_nearest_mrt.csv')

hdb_resale_df_mrt = hdb_resale_df_mrt[['town', 'flat_type', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model',
                                       'remaining_lease', 'latitude', 'longitude', 'STN_NAME', 'distance_to_mrt', 'resale_price']]

hdb_resale_df_mrt['years_left'] = hdb_resale_df_mrt['remaining_lease'].str[:2].astype(float)
hdb_resale_df_mrt['months_left'] = pd.to_numeric(hdb_resale_df_mrt['remaining_lease'].str[9:11], errors='coerce')
hdb_resale_df_mrt['months_left'].fillna(0, inplace=True)

hdb_resale_df_mrt['years_left'] = hdb_resale_df_mrt['years_left'] + hdb_resale_df_mrt['months_left']/12

hdb_resale_df_mrt.drop('months_left', inplace=True, axis=1)

street_coord_df = hdb_resale_df_mrt.groupby(['street_name', 'latitude', 'longitude'])['town'].count().to_frame().reset_index()

street_coord_df.head()

,street_name,latitude,longitude,town
0,ADMIRALTY DR,1.452055,103.820294,371
1,ADMIRALTY LINK,1.456235,103.818358,295
2,AH HOOD RD,1.327551,103.845753,13
3,ALJUNIED CRES,1.319333,103.885779,147
4,ALJUNIED RD,1.325974,103.880704,14


#### Get the nearby venues for each location

Foursquare API is used to get the venues within 1000m radius of the location which is used for the linear regression task later.

I've noticed that sometimes the API fails to connect, to ensure that all rows have data, there are multiple reattempts if the connection fails.

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        successful_bool = False
        
        # make the GET request
        for try_attempts in range(3):
            print(f'Attempt #{try_attempts+1}')
            try:
                results = requests.get(url).json()["response"]['groups'][0]['items']
                
                successful_bool = True
                break
            except:
                if try_attempts == 3:
                    successful_bool = False
                else:
                    continue

    
        if successful_bool:
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        else:
            venues_list.append([(
                name,
                lat,
                lng,
                np.nan,
                np.nan,
                np.nan,
                np.nan
                )])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
singapore_venues = getNearbyVenues(street_coord_df['street_name'], street_coord_df['latitude'], street_coord_df['longitude'], 1000)

ADMIRALTY DR
Attempt #1
ADMIRALTY LINK
Attempt #1
AH HOOD RD
Attempt #1
ALJUNIED CRES
Attempt #1
ALJUNIED RD
Attempt #1
ANCHORVALE CRES
Attempt #1
ANCHORVALE DR
Attempt #1
ANCHORVALE LANE
Attempt #1
ANCHORVALE LINK
Attempt #1
ANCHORVALE RD
Attempt #1
ANCHORVALE ST
Attempt #1
ANG MO KIO AVE 1
Attempt #1
ANG MO KIO AVE 10
Attempt #1
ANG MO KIO AVE 2
Attempt #1
ANG MO KIO AVE 3
Attempt #1
ANG MO KIO AVE 4
Attempt #1
ANG MO KIO AVE 5
Attempt #1
ANG MO KIO AVE 6
Attempt #1
ANG MO KIO AVE 8
Attempt #1
ANG MO KIO AVE 9
Attempt #1
ANG MO KIO ST 11
Attempt #1
ANG MO KIO ST 21
Attempt #1
ANG MO KIO ST 31
Attempt #1
ANG MO KIO ST 32
Attempt #1
ANG MO KIO ST 52
Attempt #1
ANG MO KIO ST 61
Attempt #1
BAIN ST
Attempt #1
BALAM RD
Attempt #1
BANGKIT RD
Attempt #1
BEACH RD
Attempt #1
BEDOK CTRL
Attempt #1
BEDOK NTH AVE 1
Attempt #1
BEDOK NTH AVE 2
Attempt #1
BEDOK NTH AVE 3
Attempt #1
BEDOK NTH AVE 4
Attempt #1
BEDOK NTH RD
Attempt #1
BEDOK NTH ST 1
Attempt #1
BEDOK NTH ST 2
Attempt #1
BEDOK NTH ST 3
A

OWEN RD
Attempt #1
PANDAN GDNS
Attempt #1
PASIR RIS DR 1
Attempt #1
PASIR RIS DR 10
Attempt #1
PASIR RIS DR 3
Attempt #1
PASIR RIS DR 4
Attempt #1
PASIR RIS DR 6
Attempt #1
PASIR RIS ST 11
Attempt #1
PASIR RIS ST 12
Attempt #1
PASIR RIS ST 13
Attempt #1
PASIR RIS ST 21
Attempt #1
PASIR RIS ST 41
Attempt #1
PASIR RIS ST 51
Attempt #1
PASIR RIS ST 52
Attempt #1
PASIR RIS ST 53
Attempt #1
PASIR RIS ST 71
Attempt #1
PASIR RIS ST 72
Attempt #1
PAYA LEBAR WAY
Attempt #1
PENDING RD
Attempt #1
PETIR RD
Attempt #1
PINE CL
Attempt #1
PIPIT RD
Attempt #1
POTONG PASIR AVE 1
Attempt #1
POTONG PASIR AVE 2
Attempt #1
POTONG PASIR AVE 3
Attempt #1
PUNGGOL CTRL
Attempt #1
PUNGGOL DR
Attempt #1
PUNGGOL EAST
Attempt #1
PUNGGOL FIELD
Attempt #1
PUNGGOL FIELD WALK
Attempt #1
PUNGGOL PL
Attempt #1
PUNGGOL RD
Attempt #1
PUNGGOL WALK
Attempt #1
PUNGGOL WAY
Attempt #1
QUEEN ST
Attempt #1
QUEEN'S CL
Attempt #1
QUEEN'S RD
Attempt #1
QUEENSWAY
Attempt #1
RACE COURSE RD
Attempt #1
REDHILL CL
Attempt #1
REDHILL LAN

In [ ]:
singapore_venues.to_csv('singapore_venues.csv')

In [6]:
singapore_venues = pd.read_csv('singapore_venues.csv')

#### Analysing the top categories for each location

We get the top 10 categories for each street and from these top 10, we further filter down to the top 20 that most commonly appear. 

This is to ensure that the code does not take ages to run.

In [8]:
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix='', prefix_sep='')

singapore_onehot['Neighbourhood'] = singapore_venues['Neighbourhood']

# move neighborhood column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

num_venues_per_area = singapore_venues.groupby(['Neighbourhood', 'Venue Category'])['Venue'].count().reset_index()

num_venues_per_area.sort_values(['Neighbourhood', 'Venue', 'Venue Category'], ascending=[True, False, True], inplace=True)

top_10_category_per_area = num_venues_per_area.groupby('Neighbourhood').head(10)

top_10_category_per_area['is_in_top_10'] =  1

singapore_grouped = singapore_onehot.groupby('Neighbourhood').mean().reset_index()


C:\Users\jimmy\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [11]:
frequent_categories = top_10_category_per_area['Venue Category'].value_counts().iloc[:20].index.tolist()

final_venue_df = pd.pivot_table(top_10_category_per_area, values='is_in_top_10', columns='Venue Category', index='Neighbourhood').fillna(0).reset_index()
final_venue_df = final_venue_df[['Neighbourhood'] + frequent_categories]

final_venue_df

Venue Category,Neighbourhood,Coffee Shop,Food Court,Chinese Restaurant,Asian Restaurant,Fast Food Restaurant,Café,Bakery,Supermarket,Bus Station,...,Park,Japanese Restaurant,Indian Restaurant,Dessert Shop,Basketball Court,Seafood Restaurant,BBQ Joint,Convenience Store,Hotel,Bus Stop
0,ADMIRALTY DR,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,ADMIRALTY LINK,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AH HOOD RD,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,ALJUNIED CRES,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,ALJUNIED RD,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,YUNG KUANG RD,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
544,YUNG LOH RD,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545,YUNG PING RD,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
546,YUNG SHENG RD,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The top 20 categories can be seen below. It seems like amenities such as Supermarket, Food Court and Coffee Shop are important for most of the streets.

In [12]:
frequent_categories

['Coffee Shop',
 'Food Court',
 'Chinese Restaurant',
 'Asian Restaurant',
 'Fast Food Restaurant',
 'Café',
 'Bakery',
 'Supermarket',
 'Bus Station',
 'Noodle House',
 'Park',
 'Japanese Restaurant',
 'Indian Restaurant',
 'Dessert Shop',
 'Basketball Court',
 'Seafood Restaurant',
 'BBQ Joint',
 'Convenience Store',
 'Hotel',
 'Bus Stop']

We can further visualize the top 10 categories below using the existing code from the course. However, it won't be used for this analysis

In [9]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [10]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = singapore_grouped['Neighbourhood']

for ind in np.arange(singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ADMIRALTY DR,Coffee Shop,Buffet,Bus Station,Fast Food Restaurant,Park,Food & Drink Shop,Shopping Mall,Fishing Spot,Bistro,Chinese Restaurant
1,ADMIRALTY LINK,Bus Station,Coffee Shop,Buffet,Fast Food Restaurant,Food Court,Sushi Restaurant,Chinese Restaurant,Fishing Spot,Seafood Restaurant,Bistro
2,AH HOOD RD,Coffee Shop,Hotel,Chinese Restaurant,Bakery,Food Court,Fast Food Restaurant,Café,Snack Place,Asian Restaurant,Sandwich Place
3,ALJUNIED CRES,Chinese Restaurant,Food Court,Noodle House,Asian Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Dim Sum Restaurant,Steakhouse,Seafood Restaurant,Coffee Shop
4,ALJUNIED RD,Chinese Restaurant,Food Court,Noodle House,Asian Restaurant,Grocery Store,Café,Coffee Shop,Indian Restaurant,Gas Station,Bakery


### Part 3 - Modelling (Linear Regression)

To determine the impact of each variable on the price of resale flats, we use linear regression. We use the existing onehot column from the venue data to join to the main table

In [13]:
hdb_resale_df_mrt_final = hdb_resale_df_mrt.merge(final_venue_df, how='left', left_on='street_name', right_on='Neighbourhood')

hdb_resale_df_mrt_final

,town,flat_type,street_name,storey_range,floor_area_sqm,flat_model,remaining_lease,latitude,longitude,STN_NAME,...,Park,Japanese Restaurant,Indian Restaurant,Dessert Shop,Basketball Court,Seafood Restaurant,BBQ Joint,Convenience Store,Hotel,Bus Stop
0,ANG MO KIO,2 ROOM,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,61 years 04 months,1.367602,103.856844,ANG MO KIO MRT STATION,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,ANG MO KIO,3 ROOM,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,60 years 07 months,1.382319,103.838706,YIO CHU KANG MRT STATION,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,ANG MO KIO,3 ROOM,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,62 years 05 months,1.376825,103.847556,YIO CHU KANG MRT STATION,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,ANG MO KIO,3 ROOM,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,62 years 01 month,1.367602,103.856844,ANG MO KIO MRT STATION,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,ANG MO KIO,3 ROOM,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,62 years 05 months,1.376825,103.847556,YIO CHU KANG MRT STATION,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82585,YISHUN,EXECUTIVE,YISHUN ST 81,04 TO 06,146.0,Maisonette,66 years 03 months,1.415253,103.837021,KHATIB MRT STATION,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82586,YISHUN,EXECUTIVE,YISHUN ST 81,01 TO 03,145.0,Maisonette,66 years 02 months,1.415253,103.837021,KHATIB MRT STATION,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82587,YISHUN,EXECUTIVE,YISHUN ST 81,07 TO 09,142.0,Apartment,66 years 02 months,1.415253,103.837021,KHATIB MRT STATION,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82588,YISHUN,MULTI-GENERATION,YISHUN ST 61,10 TO 12,164.0,Multi Generation,66 years 02 months,1.418095,103.838668,KHATIB MRT STATION,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### Making onehot columns for categorical variables 

Categorical variables such as the venue type, flat type, town names, flat model and storey range has to be dummified to be fed into the linear regression model.

To prevent the dummy variable trap from happening, 1 column is dropped to be included inside the constant variable

In [53]:
venue_df_onehot = hdb_resale_df_mrt_final[frequent_categories]

venue_df_onehot.fillna(0, inplace=True)

flat_type_onehot = pd.get_dummies(hdb_resale_df_mrt_final['flat_type'], prefix='', prefix_sep='')

flat_type_onehot.drop(['1 ROOM'], axis=1, inplace=True)

town_onehot = pd.get_dummies(hdb_resale_df_mrt_final['town'], prefix='', prefix_sep='')

town_onehot.drop(['SEMBAWANG'], axis=1, inplace=True)

flat_model_onehot = pd.get_dummies(hdb_resale_df_mrt_final['flat_model'], prefix='', prefix_sep='')

flat_model_onehot.drop(['2-room'], axis=1, inplace=True)

storey_range_onehot = pd.get_dummies(hdb_resale_df_mrt_final['storey_range'], prefix='', prefix_sep='')

storey_range_onehot.drop(['49 TO 51'], axis=1, inplace=True)

C:\Users\jimmy\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [54]:
final_df_onehot = pd.concat([town_onehot, flat_type_onehot, flat_model_onehot, venue_df_onehot], axis=1)

final_df_onehot['distance_to_mrt'] = hdb_resale_df_mrt['distance_to_mrt']

final_df_onehot['years_left'] = hdb_resale_df_mrt['years_left']

final_df_onehot['floor_area_sqm'] = hdb_resale_df_mrt['floor_area_sqm']

final_df_onehot['resale_price'] = hdb_resale_df_mrt['resale_price']

final_df_onehot

,ANG MO KIO,BEDOK,BISHAN,BUKIT BATOK,BUKIT MERAH,BUKIT PANJANG,BUKIT TIMAH,CENTRAL AREA,CHOA CHU KANG,CLEMENTI,...,Basketball Court,Seafood Restaurant,BBQ Joint,Convenience Store,Hotel,Bus Stop,distance_to_mrt,years_left,floor_area_sqm,resale_price
0,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,851.339283,61.333333,44.0,232000.0
1,1,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,696.968500,60.583333,67.0,250000.0
2,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,617.913682,62.416667,67.0,262000.0
3,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,851.339283,62.083333,68.0,265000.0
4,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,617.913682,62.416667,67.0,265000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82585,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,507.717527,66.250000,146.0,680000.0
82586,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,507.717527,66.166667,145.0,618000.0
82587,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,507.717527,66.166667,142.0,670000.0
82588,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,637.827578,66.166667,164.0,750000.0


In [55]:
X = final_df_onehot.iloc[:, :-1]

y = final_df_onehot.iloc[:, -1]

#### Modelling using statsmodels

Statsmodels is used instead of sklearn because we are able to see the impact of each individual variable on the price. 

In [56]:
import statsmodels.api as sm

X_stats = sm.add_constant(X)

stats_regr = sm.OLS(y, X_stats).fit()

stats_regr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           resale_price   R-squared:                       0.859
Model:                            OLS   Adj. R-squared:                  0.858
Method:                 Least Squares   F-statistic:                     6955.
Date:                Sun, 03 Jan 2021   Prob (F-statistic):               0.00
Time:                        21:38:07   Log-Likelihood:            -1.0229e+06
No. Observations:               82590   AIC:                         2.046e+06
Df Residuals:                   82517   BIC:                         2.047e+06
Df Model:                          72                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                  -4.799e+05   4.23e+04    -11.347      0.000   -5.63e+05   -3.97e+05
ANG MO KIO              1.906e+05   2087.502     91.289      0.000    1.86e+05    1.95e+05
BEDOK                    1.54e+05   1873.755     82.214      0.000     1.5e+05    1.58e+05
BISHAN                  2.792e+05   2246.282    124.280      0.000    2.75e+05    2.84e+05
BUKIT BATOK             1.028e+05   2139.977     48.045      0.000    9.86e+04    1.07e+05
BUKIT MERAH             2.775e+05   1986.912    139.641      0.000    2.74e+05    2.81e+05
BUKIT PANJANG           7.761e+04   1829.817     42.413      0.000     7.4e+04    8.12e+04
BUKIT TIMAH             3.665e+05   4220.055     86.857      0.000    3.58e+05    3.75e+05
CENTRAL AREA            2.465e+05   3476.640     70.895      0.000     2.4e+05    2.53e+05
CHOA CHU KANG           6247.9755   1863.973      3.352      0.001    2594.602    9901.349
CLEMENTI                2.276e+05   2170.298    104.871      0.000    2.23e+05    2.32e+05
GEYLANG                 1.987e+05   2172.831     91.456      0.000    1.94e+05    2.03e+05
HOUGANG                 9.566e+04   1836.054     52.100      0.000    9.21e+04    9.93e+04
JURONG EAST              1.18e+05   2134.039     55.310      0.000    1.14e+05    1.22e+05
JURONG WEST             4.192e+04   1733.643     24.183      0.000    3.85e+04    4.53e+04
KALLANG/WHAMPOA         2.083e+05   2240.630     92.945      0.000    2.04e+05    2.13e+05
MARINE PARADE           3.166e+05   3494.853     90.594      0.000     3.1e+05    3.23e+05
PASIR RIS               8.458e+04   1941.861     43.557      0.000    8.08e+04    8.84e+04
PUNGGOL                 6.818e+04   1905.736     35.777      0.000    6.44e+04    7.19e+04
QUEENSTOWN              3.034e+05   2173.422    139.599      0.000    2.99e+05    3.08e+05
SENGKANG                5.703e+04   1827.074     31.212      0.000    5.34e+04    6.06e+04
SERANGOON               1.794e+05   2325.175     77.139      0.000    1.75e+05    1.84e+05
TAMPINES                1.328e+05   1811.834     73.296      0.000    1.29e+05    1.36e+05
TOA PAYOH               2.026e+05   2244.539     90.275      0.000    1.98e+05    2.07e+05
WOODLANDS               2.005e+04   1692.180     11.848      0.000    1.67e+04    2.34e+04
YISHUN                   7.67e+04   1753.952     43.730      0.000    7.33e+04    8.01e+04
2 ROOM                 -4537.2634   9962.444     -0.455      0.649   -2.41e+04     1.5e+04
3 ROOM                  2.347e+04   9904.132      2.369      0.018    4053.208    4.29e+04
4 ROOM                  4.382e+04   1.01e+04      4.336      0.000     2.4e+04    6.36e+04
5 ROOM                  6.518e+04   1.04e+04      6.265      0.000    4.48e+04    8.56e+04
EXECUTIVE               6.332e+04   1.08e+04      5.858      0.000    4.21e+04  

#### Removing multi-collinearity

It seems like there are some relationship between the variables. We can use Variance Inflation Factors (VIF) to determine which variable is highly correlated to the rest. 

We first remove the highest VIF variable and then repeat the VIF analysis. Iterate the process until there are no variables with VIF > 5. 

VIF < 1 - weak correlation <br>
VIF > 1 - moderately correlated <br>
VIF > 5 - strongly correlated <br>
VIF > 10 - very strongly correlated

In [18]:
def variance_inflation_factors(exog_df):
    '''
    Parameters
    ----------
    exog_df : dataframe, (nobs, k_vars)
        design matrix with all explanatory variables, as for example used in
        regression.

    Returns
    -------
    vif : Series
        variance inflation factors
    '''
#     exog_df = sm.add_constant(exog_df)
    vifs = pd.Series(
        [1 / (1. - sm.OLS(exog_df[col].values, 
                       exog_df.loc[:, exog_df.columns != col].values).fit().rsquared) 
         for col in exog_df],
        index=exog_df.columns,
        name='VIF'
    )
    return vifs

In [57]:
X_test = X.copy()

vif_df = variance_inflation_factors(X_test).to_frame()
column_to_remove = vif_df.idxmax().values[0]
vif_value = vif_df.max().values[0]
print(f'Col to remove: {column_to_remove} {vif_value}')

# column_to_remove = 'floor_area_sqm'
run_num = 0
num_high_vif = vif_df[vif_df['VIF']>=5].shape[0]

while num_high_vif > 0:
    print(f'Run Number: {run_num}')
    print(f'Number of high VIF: {num_high_vif}')
    
    X_test = X_test.drop(column_to_remove, axis=1)
    
    print('Running VIF analysis')
    vif_df = variance_inflation_factors(X_test).to_frame()
    high_vif_df = vif_df[vif_df['VIF']>=5]
    
    num_high_vif = high_vif_df.shape[0]
    run_num += 1
    
    if num_high_vif > 0:
        column_to_remove = vif_df.idxmax().values[0]
        vif_value = vif_df.max().values[0]
        print(f'Col to remove: {column_to_remove} {vif_value}')
        
print('Finished')

C:\Users\jimmy\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars


Col to remove: MULTI-GENERATION inf
Run Number: 0
Number of high VIF: 18
Running VIF analysis
Col to remove: floor_area_sqm 360.5054323246925
Run Number: 1
Number of high VIF: 16
Running VIF analysis
Col to remove: years_left 101.14884912167219
Run Number: 2
Number of high VIF: 13
Running VIF analysis
Col to remove: Coffee Shop 16.141557273263032
Run Number: 3
Number of high VIF: 12
Running VIF analysis
Col to remove: Food Court 10.28311946381642
Run Number: 4
Number of high VIF: 11
Running VIF analysis
Col to remove: 2 ROOM 7.059744933614172
Run Number: 5
Number of high VIF: 10
Running VIF analysis
Col to remove: Fast Food Restaurant 6.799822841668376
Run Number: 6
Number of high VIF: 4
Running VIF analysis
Col to remove: Chinese Restaurant 6.259330162831208
Run Number: 7
Number of high VIF: 3
Running VIF analysis
Col to remove: distance_to_mrt 5.457875510154754
Run Number: 8
Number of high VIF: 2
Running VIF analysis
Col to remove: Asian Restaurant 5.306990242337876
Run Number: 9
Num

After iteration, it seems like the number of variables dropped significantly with the R-squared coefficient reducing to approx 0.8. 

In [58]:
import statsmodels.api as sm

X_stats = sm.add_constant(X_test)

stats_regr = sm.OLS(y, X_stats).fit()

stats_regr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           resale_price   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.793
Method:                 Least Squares   F-statistic:                     5031.
Date:                Sun, 03 Jan 2021   Prob (F-statistic):               0.00
Time:                        21:44:54   Log-Likelihood:            -1.0385e+06
No. Observations:               82590   AIC:                         2.077e+06
Df Residuals:                   82526   BIC:                         2.078e+06
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1.549e+05   4.95e+04      3.127      0.002    5.78e+04    2.52e+05
ANG MO KIO              1.563e+05   2329.084     67.091      0.000    1.52e+05    1.61e+05
BEDOK                   1.304e+05   2174.558     59.967      0.000    1.26e+05    1.35e+05
BISHAN                   2.55e+05   2516.363    101.349      0.000     2.5e+05     2.6e+05
BUKIT BATOK             7.632e+04   2365.480     32.262      0.000    7.17e+04     8.1e+04
BUKIT MERAH             2.639e+05   2213.342    119.221      0.000     2.6e+05    2.68e+05
BUKIT PANJANG           6.038e+04   2143.644     28.168      0.000    5.62e+04    6.46e+04
BUKIT TIMAH             3.328e+05   5006.348     66.477      0.000    3.23e+05    3.43e+05
CENTRAL AREA            1.889e+05   4060.345     46.520      0.000    1.81e+05    1.97e+05
CHOA CHU KANG           1116.6003   2135.049      0.523      0.601   -3068.081    5301.282
CLEMENTI                2.073e+05   2510.434     82.559      0.000    2.02e+05    2.12e+05
GEYLANG                 1.748e+05   2524.438     69.263      0.000     1.7e+05     1.8e+05
HOUGANG                 9.038e+04   2116.289     42.708      0.000    8.62e+04    9.45e+04
JURONG EAST             1.056e+05   2412.580     43.784      0.000    1.01e+05     1.1e+05
JURONG WEST             3.026e+04   2015.571     15.014      0.000    2.63e+04    3.42e+04
KALLANG/WHAMPOA         1.653e+05   2613.427     63.239      0.000     1.6e+05     1.7e+05
MARINE PARADE           2.372e+05   3953.891     59.985      0.000    2.29e+05    2.45e+05
PASIR RIS               8.782e+04   2223.134     39.502      0.000    8.35e+04    9.22e+04
PUNGGOL                  1.01e+05   2040.408     49.493      0.000     9.7e+04    1.05e+05
QUEENSTOWN               2.91e+05   2463.933    118.087      0.000    2.86e+05    2.96e+05
SENGKANG                7.713e+04   2033.833     37.924      0.000    7.31e+04    8.11e+04
SERANGOON               1.614e+05   2609.092     61.869      0.000    1.56e+05    1.67e+05
TAMPINES                1.049e+05   2076.579     50.534      0.000    1.01e+05    1.09e+05
TOA PAYOH               1.619e+05   2588.448     62.536      0.000    1.57e+05    1.67e+05
WOODLANDS               2.856e+04   1882.618     15.169      0.000    2.49e+04    3.22e+04
YISHUN                  7.441e+04   1987.363     37.444      0.000    7.05e+04    7.83e+04
3 ROOM                  7.288e+04   2037.741     35.766      0.000    6.89e+04    7.69e+04
4 ROOM                  2.006e+05   2016.603     99.452      0.000    1.97e+05    2.05e+05
5 ROOM                   3.48e+05   2075.758    167.653      0.000    3.44e+05    3.52e+05
EXECUTIVE               3.586e+05   3307.886    108.404      0.000    3.52e+05    3.65e+05
Adjoined flat          -1.412e+04   4.99e+04     -0.283      0.777   -1.12e+05  

We export the summary to a CSV file for further preprocessing

In [59]:
f = open('stats.csv', 'w')

f.write(stats_regr.summary().as_csv())
f.close()